## Overview

This notebook shows how to use Instructor with LLMs to generate simple Python code. It uses validation to ensure the dependencies are correct and that the code runs without errors.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.

In [6]:
%%capture
!pip install instructor groq openai

On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY

GROQ_API_KEY

In [8]:
import instructor
import openai
import groq
from pydantic import BaseModel, Field, field_validator
from typing import List
import subprocess
from rich import print as rprint
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
    os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')
except Exception as e:
    # print(e)
    pass


Now to the cool stuff...

In [9]:
inference_provider = "openai"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())

class CodeFile(BaseModel):
    """
    Python code and dependencies to meet specified requirements. Dependencies will be installed and the code will be executed.

    Limitations:
    - This code will not be tested for quality, only that it runs without errors.
    """
    architecture: str = Field(description="A markdown file that describes the architecture of the code and how it will satisfy each requirement one by one.")
    dependencies: List[str] = Field(description="List of pip packages required to run the code.")
    code: str = Field(description="A single Python file.")

    # the order above specifies the order of validation

    @field_validator('dependencies')
    @classmethod
    def validate_dependencies(cls, v):
        if v:
            try:
                subprocess.run(["pip", "install"] + v, check=True)
                print("Dependencies installed: " + ", ".join(v))
                print("\n\n")
            except subprocess.CalledProcessError as e:
                error_value = "Failed to install dependencies: " + str(e)
                print(error_value)
                raise ValueError(error_value)
        return v

    @field_validator('code')
    @classmethod
    def validate_code(cls, v):
        if v:
            try:
                temp_code = v
                print("Python code\n===========\n")
                rprint(temp_code)
                print("\n\nTrying code\n===========\n")
                subprocess.run(["python", "-c", temp_code])
            except subprocess.CalledProcessError as e:
                error_value = "Error while executing code: " + str(e)
                print(error_value)
                raise ValueError(error_value)
        return v

result = client.chat.completions.create(
    model="llama3-70b-8192" if inference_provider == "groq" else "gpt-4-turbo",
    response_model=CodeFile,
    temperature=0.0,
    max_retries=3,
    messages=[{"role": "user", "content": """
        Extract the text content of https://example.com as markdown and print it out.
        """}]
    )

code = result.code
# rprint(code)

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached markdownify-0.12.1-py3-none-any.whl.metadata (8.0 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached markdownify-0.12.1-py3-none-any.whl (16 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl (119 kB)
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
Dependencies installed: requests, markdownify



Python code



import requests
from markdownify import markdownify as md

# URL to extract content from
url = 'https://example.com'

# Send HTTP request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert HTML to Markdown
    markdown_text = md(response.text)
    print(markdown_text)
else:
    print(f'Failed to retrieve content. Status code: {response.status_code}')



Trying code




Example Domain







Example Domain


This domain is for use in illustrative examples in documents. You may use this
 domain in literature without prior coordination or asking for permission.


[More information...](https://www.iana.org/domains/example)






